In [1]:
import numpy as np
import pandas as pd
import nltk
from fuzzywuzzy import fuzz

/home/itm3f/.local/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
##Function that takes in 3 lists, the list of strings you want to compare, the list of strings you're comparing to and a list with the original way the string from list 1 was written.  
def matchWithFuzzyNames(l1, l2, original, row_num):
    match = pd.DataFrame(columns = ['original company names', 'clean company name', 'company matches', 'fuzz ratio'])
    for i in l1:
        matches = []
        score = []
        score_partial = []
        for j in l2:
            #due to time constraints, we set the fuzz.ratio to 100, however future programming should adjust this value to some other quantity to see if we get better matches
            if (fuzz.ratio(i,j)) == 100:
                matches.append(j)
                score.append(fuzz.ratio(i,j))
                #the partial ratio will return the ratio raw score of 100  if the shorter string is found an any of substrings of the larger string, used this as a safety net and potentially as an additional step to 
                score_partial.append((fuzz.partial_ratio(i,j)))
        match = match.append({ 'clean company name': i,'company matches': matches, 'fuzz ratio': score, 'fuzz partial ratio': score_partial},ignore_index=True)
        
    match['original company names'] = original
    match['original row number'] = row_num
 
#Will output a pandas dataframe with 4 columns
    return match

In [13]:
#Isabels function; I added List at the end to differentiate between this and the top function
#I am using this original function to do DNA
def matchWithFuzzyNamesList(l1, l2, original):
    match = pd.DataFrame(columns = ['original company names', 'clean company name', 'company matches', 'fuzz ratio'])
    for i in l1:
        matches = []
        score = []
        score_partial = []
        for j in l2:
            if (fuzz.ratio(i,j)) == 100:
                matches.append(j)
                score.append(fuzz.ratio(i,j))
                score_partial.append((fuzz.partial_ratio(i,j)))
        match = match.append({ 'clean company name': i,'company matches': matches, 'fuzz ratio': score, 'fuzz partial ratio': score_partial},ignore_index=True)
        
    match['original company names'] = original
        

 


    return match

<h2>Replicating Isabel's code</h2>

In [15]:
fda = pd.read_csv("../data/working/fda_clean.csv", index_col = [0])

In [17]:
fda.columns

Index(['FDA Companies ', 'Company Clean'], dtype='object')

In [18]:
fda_clean = fda.dropna()

In [113]:
fda.head()

,FDA Companies,Company Clean
0,3D IMAGING DRUG,d imaging drug
1,3M,m
2,3M DRUG DELIVERY,m drug delivery
3,AAIPHARMA LLC,aaipharma
4,ABBOTT LABS,abbott


In [21]:
fda_org = fda_clean['FDA Companies '].reset_index()

In [22]:
fda_org

,index,FDA Companies
0,0,3D IMAGING DRUG
1,1,3M
2,2,3M DRUG DELIVERY
3,3,AAIPHARMA LLC
4,4,ABBOTT LABS
...,...,...
968,970,ZO SKIN HEALTH
969,971,ZYDUS HLTHCARE
970,972,ZYDUS PHARMS USA
971,973,ZYDUS PHARMS USA INC


In [23]:
fda_row = fda_org['index']

In [24]:
fda_row

0        0
1        1
2        2
3        3
4        4
      ... 
968    970
969    971
970    972
971    973
972    974
Name: index, Length: 973, dtype: int64

In [26]:
fda_comp = fda_org['FDA Companies ']

In [27]:
fda_comp

0           3D IMAGING DRUG
1                        3M
2          3M DRUG DELIVERY
3             AAIPHARMA LLC
4               ABBOTT LABS
               ...         
968          ZO SKIN HEALTH
969          ZYDUS HLTHCARE
970        ZYDUS PHARMS USA
971    ZYDUS PHARMS USA INC
972         ZYDUS WORLDWIDE
Name: FDA Companies , Length: 973, dtype: object

In [28]:
fda_list = fda_clean['Company Clean']

In [29]:
fda_list

0       d imaging drug
1                    m
2      m drug delivery
3            aaipharma
4               abbott
            ...       
970            zo skin
971              zydus
972              zydus
973              zydus
974    zydus worldwide
Name: Company Clean, Length: 973, dtype: object

In [31]:
matching = matchWithFuzzyNames(fda_list, fda_list, fda_comp, fda_row)

In [32]:
matching

,original company names,clean company name,company matches,fuzz ratio,fuzz partial ratio,original row number
0,3D IMAGING DRUG,d imaging drug,[d imaging drug],[100],[100],0
1,3M,m,[m],[100],[100],1
2,3M DRUG DELIVERY,m drug delivery,[m drug delivery],[100],[100],2
3,AAIPHARMA LLC,aaipharma,[aaipharma],[100],[100],3
4,ABBOTT LABS,abbott,[abbott],[100],[100],4
...,...,...,...,...,...,...
968,ZO SKIN HEALTH,zo skin,[zo skin],[100],[100],970
969,ZYDUS HLTHCARE,zydus,"[zydus, zydus, zydus]","[100, 100, 100]","[100, 100, 100]",971
970,ZYDUS PHARMS USA,zydus,"[zydus, zydus, zydus]","[100, 100, 100]","[100, 100, 100]",972
971,ZYDUS PHARMS USA INC,zydus,"[zydus, zydus, zydus]","[100, 100, 100]","[100, 100, 100]",973


In [33]:
#original row number is the row number from fda_clean before the dropna()s

In [35]:
matching['company matches'] = matching['company matches'].astype(str)
matching['original row number'] = matching['original row number'].astype(str)
matching['fuzz ratio'] = matching['fuzz ratio'].astype(str)

In [36]:
#group by clean company name, append the original company names, row numbers and ratio value to new colum
unique_comp = matching.groupby('clean company name', as_index = False).agg({'original company names': ', '.join, 
                                                                            'original row number': ', '.join,
                                                                            'company matches': ', '.join,
                                                                           'fuzz ratio': ', '.join} )

In [37]:
unique_comp

,clean company name,original company names,original row number,company matches,fuzz ratio
0,aaipharma,AAIPHARMA LLC,3,['aaipharma'],[100]
1,abbott,ABBOTT LABS,4,['abbott'],[100]
2,abbvie,"ABBVIE, ABBVIE INC","5, 8","['abbvie', 'abbvie'], ['abbvie', 'abbvie']","[100, 100], [100, 100]"
3,abbvie endocrine,"ABBVIE ENDOCRINE, ABBVIE ENDOCRINE INC","6, 7","['abbvie endocrine', 'abbvie endocrine'], ['ab...","[100, 100], [100, 100]"
4,abhai,"ABHAI INC, ABHAI LLC","9, 10","['abhai', 'abhai'], ['abhai', 'abhai']","[100, 100], [100, 100]"
...,...,...,...,...,...
791,zambon,ZAMBON SPA,968,['zambon'],[100]
792,zevacor,ZEVACOR PHARMA INC,969,['zevacor'],[100]
793,zo skin,ZO SKIN HEALTH,970,['zo skin'],[100]
794,zydus,"ZYDUS HLTHCARE, ZYDUS PHARMS USA, ZYDUS PHARMS...","971, 972, 973","['zydus', 'zydus', 'zydus'], ['zydus', 'zydus'...","[100, 100, 100], [100, 100, 100], [100, 100, 100]"


In [38]:
og = pd.read_excel("../data/original/fda_companies.xlsx")

In [39]:
og.head()

,FDA Companies
0,3D IMAGING DRUG
1,3M
2,3M DRUG DELIVERY
3,AAIPHARMA LLC
4,ABBOTT LABS


<h3>Using Isabel's Method on NDC</h3>

In [55]:
ndc = pd.read_csv("../data/working/ndc_clean.csv", index_col = [0])

In [56]:
ndc_clean = ndc.dropna()

In [57]:
ndc.columns

Index(['original_company', 'originalRow', 'cleaned_name'], dtype='object')

In [58]:
ndc_org = ndc_clean['original_company'].reset_index()

In [59]:
ndc_org

,row,original_company
0,0,SPIRONOLACTONE 2%
1,1,-L'Oreal USA Products Inc
2,2,.Cardinal Health
3,3,.Church & Dwight Canada Corp
4,4,{Preferred Pharmaeutials INC.
...,...,...
6712,7018,Zydus Pharmaceuticals USA Inc
6713,7019,Zydus Pharmaceuticals USA Inc.
6714,7020,Zydus Technologies Limited
6715,7021,ZYGONE


In [60]:
ndc_row = ndc_org['row']

In [61]:
ndc_comp = ndc_org['original_company']

In [62]:
ndc_list = ndc_clean['cleaned_name']

In [64]:
matching = matchWithFuzzyNames(ndc_list, ndc_list, ndc_comp, ndc_row)

In [65]:
matching

,original company names,clean company name,company matches,fuzz ratio,fuzz partial ratio,original row number
0,SPIRONOLACTONE 2%,spironolactone,[spironolactone],[100],[100],0
1,-L'Oreal USA Products Inc,-loreal,[-loreal],[100],[100],1
2,.Cardinal Health,cardinal,"[cardinal, cardinal, cardinal, cardinal, cardi...","[100, 100, 100, 100, 100, 100, 100, 100, 100, ...","[100, 100, 100, 100, 100, 100, 100, 100, 100, ...",2
3,.Church & Dwight Canada Corp,church&dwight canada,"[church&dwight canada, church&dwight canada, c...","[100, 100, 100]","[100, 100, 100]",3
4,{Preferred Pharmaeutials INC.,preferred pharmaeutials,[preferred pharmaeutials],[100],[100],4
...,...,...,...,...,...,...
6712,Zydus Pharmaceuticals USA Inc,zydus,"[zydus, zydus, zydus, zydus]","[100, 100, 100, 100]","[100, 100, 100, 100]",7018
6713,Zydus Pharmaceuticals USA Inc.,zydus,"[zydus, zydus, zydus, zydus]","[100, 100, 100, 100]","[100, 100, 100, 100]",7019
6714,Zydus Technologies Limited,zydus,"[zydus, zydus, zydus, zydus]","[100, 100, 100, 100]","[100, 100, 100, 100]",7020
6715,ZYGONE,zygone,[zygone],[100],[100],7021


In [95]:
matching['company matches'] = matching['company matches'].astype(str)
matching['original row number'] = matching['original row number'].astype(str)
matching['fuzz ratio'] = matching['fuzz ratio'].astype(str)
matching['original row number'] = (matching['original row number'].astype(int) + 25).astype(str) #adding 25 because 25 numbers were removed from the og dataset before cleaning, so original row ended up being 25 less than it was suppose to

In [96]:
matching

,original company names,clean company name,company matches,fuzz ratio,fuzz partial ratio,original row number,og
0,SPIRONOLACTONE 2%,spironolactone,['spironolactone'],[100],[100],25,25
1,-L'Oreal USA Products Inc,-loreal,['-loreal'],[100],[100],26,26
2,.Cardinal Health,cardinal,"['cardinal', 'cardinal', 'cardinal', 'cardinal...","[100, 100, 100, 100, 100, 100, 100, 100, 100, ...","[100, 100, 100, 100, 100, 100, 100, 100, 100, ...",27,27
3,.Church & Dwight Canada Corp,church&dwight canada,"['church&dwight canada', 'church&dwight canada...","[100, 100, 100]","[100, 100, 100]",28,28
4,{Preferred Pharmaeutials INC.,preferred pharmaeutials,['preferred pharmaeutials'],[100],[100],29,29
...,...,...,...,...,...,...,...
6712,Zydus Pharmaceuticals USA Inc,zydus,"['zydus', 'zydus', 'zydus', 'zydus']","[100, 100, 100, 100]","[100, 100, 100, 100]",7043,7043
6713,Zydus Pharmaceuticals USA Inc.,zydus,"['zydus', 'zydus', 'zydus', 'zydus']","[100, 100, 100, 100]","[100, 100, 100, 100]",7044,7044
6714,Zydus Technologies Limited,zydus,"['zydus', 'zydus', 'zydus', 'zydus']","[100, 100, 100, 100]","[100, 100, 100, 100]",7045,7045
6715,ZYGONE,zygone,['zygone'],[100],[100],7046,7046


In [97]:
unique_comp = matching.groupby('clean company name', as_index = False).agg({'original company names': ', '.join, 
                                                                            'original row number': ', '.join,
                                                                            'company matches': ', '.join,
                                                                           'fuzz ratio': ', '.join} )

In [116]:
unique_comp.tail(50)

,clean company name,original company names,original row number,company matches,fuzz ratio
5159,yung shin ind,"Yung Shin Pharmaceutical Ind. Co., Ltd.",6990,['yung shin ind'],[100]
5160,yuyao jessie commodity,"Yuyao Jessie Commodity Co.,Ltd.",6991,['yuyao jessie commodity'],[100]
5161,yuyao jessie promotional,"Yuyao Jessie Promotional Products Co., Ltd.",6992,['yuyao jessie promotional'],[100]
5162,yves rocher amerique du nord,Yves Rocher Amerique du Nord Inc,6993,['yves rocher amerique du nord'],[100]
5163,yves rocher north america,Yves Rocher North America Inc.,6994,['yves rocher north america'],[100]
5164,zag,Zag Usa Inc,6995,['zag'],[100]
5165,zaol,zaol,6996,['zaol'],[100]
5166,zapatol,Zapatol Corp.,6997,['zapatol'],[100]
5167,zapne,Zapne LLC,6998,['zapne'],[100]
5168,zapotol,Zapotol Corp.,6999,['zapotol'],[100]


In [84]:
og = pd.read_excel("../data/original/BI DSPG Company Datasets/NDC_Company_Dataset.xls")

In [117]:
#because I deleted the first 25 rows (the original row number was 25 less than it was suppose to be, so I added 25 to the og row number above to accurately reflect the actual row the company was in prior to any deletions)
og.iloc[7010]


Row Labels    Zermat Internacional
Name: 7010, dtype: object

<h2>DNA Matching</h2>

In [3]:
dna = pd.read_csv("../data/working/dna_clean.csv", index_col = [0])

In [5]:
del dna['Unnamed: 0.1']
del dna['Code']

In [6]:
dna.dropna(inplace = True)

In [7]:
dna.reset_index(inplace = True)

In [8]:
dna

,index,Description,cleaned_companies
0,0,AA PLC,aa
1,1,"Emperial Americas, Inc.",emperial americas
2,2,"American Academy of Allergy, Asthma and Immuno...",academy allergy asthma immunology
3,3,Bird Studies Canada,bird studies canada
4,4,Aesculap AG & Co. KG,aesculap&co
...,...,...,...
63949,64000,Boost Brent Oil 3x Leverage Daily ETP,boost brent oil x leverage daily etp
63950,64001,Boost Brent Oil 3x Short Daily ETP,boost brent oil x short daily etp
63951,64002,Nuveen High Income November 2021 Target Term Fund,nuveen high income november target term fund
63952,64003,SPDR SSGA Gender Diversity Index ETF,spdr ssga gender diversity index etf


In [9]:
new_original_company = dna['Description']

In [11]:
new_original_company.tail()

63949                Boost Brent Oil 3x Leverage Daily ETP
63950                   Boost Brent Oil 3x Short Daily ETP
63951    Nuveen High Income November 2021 Target Term Fund
63952                 SPDR SSGA Gender Diversity Index ETF
63953               Health Insurance Institute of Slovenia
Name: Description, dtype: object

In [ ]:
#Running Isabels code
dna_matching = matchWithFuzzyNamesList(dna['cleaned_companies'].tolist(), dna['cleaned_companies'].tolist(), new_original_company)